In [1]:
import pandas as pd
import numpy as np

In [2]:
##load data
polypred_max1 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max1.prs",sep='\t')
polypred_max3 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max3.prs",sep='\t')
polypred_max5 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max5.prs",sep='\t')
polypred_max7 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max7.prs",sep='\t')
polypred_max10 = pd.read_csv("/gpfs/commons/home/tlin/output/bellenguez/bellenguez_all_2/polypred/polypred_bellenguez_max10.prs",sep='\t')
manifest = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/SampleManifest_DS_2021.02.19_ALL.txt", sep='\t') ## the one with mapping info
phenotype1 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADNIPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype2 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSPCaseControlPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype3 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/ADSPFamilyBasedPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')
phenotype4 = pd.read_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/PSPCBDPhenotypes_DS_2021.02.19_ALL.txt", sep='\t')

### checking if there are any unmapped IID in the polypred result

In [3]:
polypred_max10[~polypred_max10.IID.isin(manifest.SampleID)] ## all the sample ID in the ADSP plink file are found  in the manifest file

,FID,IID,PRS


In [34]:
phenotype3['diagnosis'] = 0

In [38]:
phenotype3.loc[:,'diagnosis'][(phenotype3.AD<= 3) & (phenotype3.AD > 0)]=1

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [39]:
phenotype3[(phenotype3.AD <=3) & (phenotype3.AD >0)].loc[:,'diagnosis']

0        1
1        1
2        1
3        1
5        1
        ..
12305    1
12308    1
12309    1
12319    1
12322    1
Name: diagnosis, Length: 1844, dtype: int64

In [40]:
phenotype3["diagnosis"].sum()

1844

In [ ]:
#test duplicates
concat = pd.concat([phenotype3, Diagnosis])   
duplicatedID=concat[concat.SUBJID.duplicated()].SUBJID 
check_diagnosis= concat[concat.SUBJID.isin(duplicatedID)].loc[:, ("SUBJID","Diagnosis")] 
check_diagnosis=check_diagnosis.sort_values(by='SUBJID')   

In [ ]:
 for i in range(1,113):                                                                                                                                                             
...     check =int(check_diagnosis.Diagnosis[i-1:i])-int(check_diagnosis.Diagnosis[i:i+1])                                                                                                
...     if(check !=0):                                                                                                                                                                  
...             print(check_diagnosis.SUBJID[i-1:i])      

In [14]:
subjid = manifest[manifest.SampleID.isin(polypred_max10.IID)].SUBJID  ## map Sample ID to Subject ID
subjid

20503    A-LOAD-LD001510
20504    A-LOAD-LD003528
20505    A-LOAD-LD004558
20506    A-LOAD-LD005876
20507    A-LOAD-LD005879
              ...       
37404     A-MIA-UM010214
37405     A-MIA-UM010240
37406     A-MIA-UM000154
37407     A-ADC-AD005530
37408     A-ADC-AD005802
Name: SUBJID, Length: 16906, dtype: object

In [47]:
phenotype=pd.concat([phenotype1, phenotype2, phenotype3, phenotype4])
phenotype[phenotype.SUBJID.isin(subjid)].shape[0] ## total: 16811
phenotype=phenotype[phenotype.SUBJID.isin(subjid)]
ID = phenotype.SUBJID.drop_duplicates()

In [6]:
found = manifest[manifest.SUBJID.isin(ID)].SampleID ## map Subject ID back to Sample ID
found.shape ## (20033, ) it is not 1-1 mapping!
polypred_max10[~polypred_max10.IID.isin(found)]   ## All ID were found!

,FID,IID,PRS


### Create diagnosis column

In [45]:
## because phenotype3 and phenotype4 dont have PrevAD and IncAD, so I only considered phenotype1 and phenotype2

phenotype=pd.concat([phenotype1, phenotype2])
#phenotype[phenotype.SUBJID.isin(subjid)].shape[0] ## total: 13932
#phenotype=phenotype[phenotype.SUBJID.isin(subjid)]
#ID = phenotype.SUBJID.drop_duplicates() ## There is no duplicates in subject ID

In [46]:
phenotype = phenotype.dropna(subset = ["PrevAD",'IncAD'])
phenotype["diagnosis"]= phenotype[['PrevAD', 'IncAD']].max(axis=1).astype(int)


In [ ]:
phenotype.merge(phenotype3, on="diagnosis")

In [47]:

ID = phenotype.SUBJID.drop_duplicates() ## There is no duplicates in subject ID

NameError: name 'diagnosis' is not defined

### Check if there are conflict  
#### phenotype 1 & 2 (not finished)

In [33]:
phenotype=pd.concat([phenotype1, phenotype2])
def merge_polypred_phenotype(polypred, pheno=phenotype, manifest=manifest):
    subjid = manifest[manifest.SampleID.isin(polypred.IID)].SUBJID  ## map Sample ID to Subject ID
    pheno=pheno[pheno.SUBJID.isin(subjid)]
    print("pheno shape = ",pheno.shape[0])
    ID = pheno.SUBJID.drop_duplicates() ##SubjID 
    found = manifest[manifest.SUBJID.isin(ID)].SampleID ## map Subject ID back to Sample ID
    print("are there any unmapped polypred ID?")
    print("polypred shape = ", polypred.shape, "; not-matching shape = ", polypred[~polypred.IID.isin(found)].shape) 
    print("pheno before removing NA in diagnosis= ", pheno.shape)
    pheno = pheno.dropna(subset = ["PrevAD",'IncAD'])
    print("pheno after removing NA in diagnosis = ", pheno.shape)
    pheno["Diagnosis"]= pheno[['PrevAD', 'IncAD']].max(axis=1).astypex(int)

    
    reverse_map = manifest[manifest.SUBJID.isin(ID)]
    polypred.rename({"IID":"SampleID"}, axis=1, inplace=True)
    info =pd.merge(pd.merge(polypred, reverse_map[reverse_map.SampleID.isin(polypred.SampleID)][["SUBJID","SampleID"]]),pheno).loc[:,["SampleID","PRS","SUBJID","Sex","Diagnosis"]]
    return(info)


In [35]:
max1 = merge_polypred_phenotype(polypred_max1)
max3 = merge_polypred_phenotype(polypred_max3)
max5 = merge_polypred_phenotype(polypred_max5)
max7 = merge_polypred_phenotype(polypred_max7)
max10 = merge_polypred_phenotype(polypred_max10)


pheno shape =  13932
are there any unmapped polypred ID?
polypred shape =  (16906, 3) ; not-matching shape =  (2914, 3)
pheno before removing NA in diagnosis=  (13932, 36)
pheno after removing NA in diagnosis =  (10948, 36)
pheno shape =  13932
are there any unmapped polypred ID?
polypred shape =  (16906, 3) ; not-matching shape =  (2914, 3)
pheno before removing NA in diagnosis=  (13932, 36)
pheno after removing NA in diagnosis =  (10948, 36)
pheno shape =  13932
are there any unmapped polypred ID?
polypred shape =  (16906, 3) ; not-matching shape =  (2914, 3)
pheno before removing NA in diagnosis=  (13932, 36)
pheno after removing NA in diagnosis =  (10948, 36)
pheno shape =  13932
are there any unmapped polypred ID?
polypred shape =  (16906, 3) ; not-matching shape =  (2914, 3)
pheno before removing NA in diagnosis=  (13932, 36)
pheno after removing NA in diagnosis =  (10948, 36)
pheno shape =  13932
are there any unmapped polypred ID?
polypred shape =  (16906, 3) ; not-matching sha

In [41]:
max1.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_1.tsv",index=False, sep = '\t')
max3.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_3.tsv",index=False, sep = '\t')
max5.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_5.tsv",index=False, sep = '\t')
max7.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_7.tsv",index=False, sep = '\t')
max10.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp10.tsv",index=False, sep = '\t')

In [25]:
phenotype

,SUBJID,Sex,PrevAD,IncAD,Age_current,Age_baseline,Age_MCI_onset,Age_AD_onset,APOE,AUTOPSY,...,Update_baseline,Update_latest,Update_Diagnosis,Correction,Age,Selection,AD,Update_Baseline,Update_Latest,Update_ADstatus
0,ADNI_002_S_0413,1,0.0,0.0,87,76,NaN,NaN,33.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
1,ADNI_002_S_0685,1,0.0,0.0,90+,89,NaN,NaN,33.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
2,ADNI_002_S_0729,1,0.0,1.0,72,65,NaN,68,34.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
3,ADNI_002_S_1155,0,0.0,0.0,69,57,52,NaN,33.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,ADNI_002_S_1261,1,0.0,0.0,82,71,81,NaN,33.0,0.0,...,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27542,A-MIA-UM001127,0,1.0,0.0,NaN,78,NaN,NaN,34.0,NaN,...,NaN,NaN,NaN,0,73,NaN,1.0,0.0,1.0,0.0
27543,A-MIA-UM001646,1,1.0,0.0,NaN,78,NaN,NaN,34.0,NaN,...,NaN,NaN,NaN,0,68,NaN,1.0,0.0,1.0,0.0
27544,A-MIA-UM000037,1,1.0,0.0,NaN,87,NaN,NaN,34.0,NaN,...,NaN,NaN,NaN,0,74,NaN,1.0,0.0,1.0,0.0
27545,A-MIA-UM000880,1,1.0,0.0,NaN,78,NaN,NaN,34.0,NaN,...,NaN,NaN,NaN,0,70,NaN,1.0,0.0,1.0,0.0


In [102]:
reverse_map = manifest[manifest.SUBJID.isin(ID)]

In [103]:
reverse_map[reverse_map.SUBJID.duplicated()].SUBJID

12883     A-MIA-UM000329
12886     A-MIA-UM000336
12910     A-MIA-UM000784
12920     A-MIA-UM001640
12941     A-MIA-UM001891
              ...       
36868    G-MSBB-MB000175
37398     A-MIA-UM001948
37399     A-MIA-UM000866
37401     A-MIA-UM000037
37403     A-MIA-UM001640
Name: SUBJID, Length: 2973, dtype: object

In [104]:
reverse_map[reverse_map.SUBJID.sort_values().duplicated()] #A-ACT-AC000014

/gpfs/commons/home/tlin/.conda/envs/polyfun/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
63,A-ACT-AC000158,A-ACT-AC000158-BL-UPN-13998,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
81,A-ACT-AC000196,A-ACT-AC000196-BL-UWA-13874,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
85,A-ACT-AC000208,A-ACT-AC000208-BL-UPN-8382,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
103,A-ACT-AC000288,A-ACT-AC000288-BL-UPN-33628,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
106,A-ACT-AC000292,A-ACT-AC000292-BL-UPN-14599,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
...,...,...,...,...,...,...,...,...,...,...
36739,G-MSBB-MB000022,G-MSBB-MB000022-BR-MSBB-71813,Brain,DNA,NYGC,PCRFree_HiSeqX,WGS,0,sa000011,snd10011
36837,G-MSBB-MB000126,G-MSBB-MB000126-BR-MSBB-71926,Brain,DNA,NYGC,PCRFree_HiSeqX,WGS,0,sa000011,snd10011
36868,G-MSBB-MB000175,G-MSBB-MB000175-BR-MSBB-71959,Brain,DNA,NYGC,PCRFree_HiSeqX,WGS,0,sa000011,snd10011
37398,A-MIA-UM001948,A-MIA-UM001948-BR-MIA-201307290uf,Brain,DNA,USUHS,PCRFree_Novaseq,WGS,0,sa000001,snd10020


In [146]:
16780+3253

20033

In [105]:
reverse_map[reverse_map.SUBJID == 'A-ACT-AC000158']

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set
63,A-ACT-AC000158,A-ACT-AC000158-BL-UPN-13998,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0,sa000001,snd10000
21395,A-ACT-AC000158,A-ACT-AC000158-BL-NCR-15AD77996,Blood,DNA,Broad,Illumina HiSeq X Ten,WGS,0,sa000001,snd10001


## drow dot plot

In [108]:
polypred_max10

,FID,IID,PRS
0,A-LOAD-LD001510-BL-NCR-06AD9047,A-LOAD-LD001510-BL-NCR-06AD9047,-0.00294
1,A-LOAD-LD003528-BR-NCR-03AD4945,A-LOAD-LD003528-BR-NCR-03AD4945,-0.00268
2,A-LOAD-LD004558-BL-NCR-04AD6349,A-LOAD-LD004558-BL-NCR-04AD6349,-0.00259
3,A-LOAD-LD005876-BL-NCR-04AD6143,A-LOAD-LD005876-BL-NCR-04AD6143,-0.00296
4,A-LOAD-LD005879-BL-NCR-04AD6161,A-LOAD-LD005879-BL-NCR-04AD6161,-0.00280
...,...,...,...
16901,A-MIA-UM010214-BR-MIA-201400626,A-MIA-UM010214-BR-MIA-201400626,-0.00251
16902,A-MIA-UM010240-BR-MIA-201601654,A-MIA-UM010240-BR-MIA-201601654,-0.00265
16903,A-MIA-UM000154-BR-MIA-201601652,A-MIA-UM000154-BR-MIA-201601652,-0.00277
16904,A-ADC-AD005530-BL-NCR-09AD18335,A-ADC-AD005530-BL-NCR-09AD18335,-0.00254


In [114]:
reverse_map[reverse_map.SampleID.isin(polypred_max10.IID)][["SUBJID","SampleID"]]

,SUBJID,SampleID
20516,A-MIA-UM000037,A-MIA-UM000037-BR-MIA-19890652
20519,A-MIA-UM000329,A-MIA-UM000329-BL-MIA-19900942
20520,A-MIA-UM000336,A-MIA-UM000336-BR-MIA-201307289
20522,A-MIA-UM000463,A-MIA-UM000463-BR-MIA-201307288
20523,A-MIA-UM000474,A-MIA-UM000474-BR-MIA-201307282
...,...,...
37400,A-MIA-UM005927,A-MIA-UM005927-BR-MIA-200521579
37401,A-MIA-UM000037,A-MIA-UM000037-BR-MIA-201307277
37402,A-MIA-UM001230,A-MIA-UM001230-BR-MIA-200427223
37403,A-MIA-UM001640,A-MIA-UM001640-BR-MIA-201335875


In [121]:
polypred_max10[polypred_max10.IID.isin(pd.concat([reverse_map,diagnosis]))]

,FID,IID,PRS


In [122]:
pd.concat([reverse_map,diagnosis])

,SUBJID,SampleID,BODY_SITE,ANALYTE_TYPE,Sequencing_Center,Sequencing_Platform,SAMPLE_USE,Technical_Replicate,Study_DSS,Sample_Set,diagnosis
4,A-ACT-AC000014,A-ACT-AC000014-BL-UPN-23967,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0.0,sa000001,snd10000,NaN
13,A-ACT-AC000034,A-ACT-AC000034-BL-UPN-15865,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0.0,sa000001,snd10000,NaN
26,A-ACT-AC000057,A-ACT-AC000057-BL-UPN-14863,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0.0,sa000001,snd10000,NaN
31,A-ACT-AC000072,A-ACT-AC000072-BL-UPN-7532,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0.0,sa000001,snd10000,NaN
38,A-ACT-AC000088,A-ACT-AC000088-BL-UPN-13977,Blood,DNA,WashU,Illumina HiSeq 2000,WES,0.0,sa000001,snd10000,NaN
...,...,...,...,...,...,...,...,...,...,...,...
27542,A-MIA-UM001127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
27543,A-MIA-UM001646,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
27544,A-MIA-UM000037,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
27545,A-MIA-UM000880,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0


In [129]:
polypred_max10[polypred_max10.IID.isin(reverse_map.SampleID)]

,FID,IID,PRS
13,A-MIA-UM000037-BR-MIA-19890652,A-MIA-UM000037-BR-MIA-19890652,-0.00239
16,A-MIA-UM000329-BL-MIA-19900942,A-MIA-UM000329-BL-MIA-19900942,-0.00301
17,A-MIA-UM000336-BR-MIA-201307289,A-MIA-UM000336-BR-MIA-201307289,-0.00266
19,A-MIA-UM000463-BR-MIA-201307288,A-MIA-UM000463-BR-MIA-201307288,-0.00216
20,A-MIA-UM000474-BR-MIA-201307282,A-MIA-UM000474-BR-MIA-201307282,-0.00240
...,...,...,...
16897,A-MIA-UM005927-BR-MIA-200521579,A-MIA-UM005927-BR-MIA-200521579,-0.00276
16898,A-MIA-UM000037-BR-MIA-201307277,A-MIA-UM000037-BR-MIA-201307277,-0.00240
16899,A-MIA-UM001230-BR-MIA-200427223,A-MIA-UM001230-BR-MIA-200427223,-0.00264
16900,A-MIA-UM001640-BR-MIA-201335875,A-MIA-UM001640-BR-MIA-201335875,-0.00272


In [134]:
polypred_max10.rename({"IID":"SampleID"}, axis=1, inplace=True)

In [148]:
info =pd.merge(pd.merge(polypred_max10, reverse_map[reverse_map.SampleID.isin(polypred_max10.SampleID)][["SUBJID","SampleID"]]),phenotype).loc[:,["SampleID","PRS","SUBJID","diagnosis"]]

In [150]:
info.to_csv("/gpfs/commons/groups/knowles_lab/data/ADSP_reguloML/ADSP_vcf/compact_filtered_vcf_16906/phenotype_data/diagnosis/prs_diagnosis_0219.2021_max_snp_10.tsv",index=False, sep = '\t')

In [151]:
info

,SampleID,PRS,SUBJID,diagnosis
0,A-MIA-UM000037-BR-MIA-19890652,-0.00239,A-MIA-UM000037,1
1,A-MIA-UM000037-BR-MIA-201307277,-0.00240,A-MIA-UM000037,1
2,A-MIA-UM000329-BL-MIA-19900942,-0.00301,A-MIA-UM000329,1
3,A-MIA-UM000336-BR-MIA-201307289,-0.00266,A-MIA-UM000336,1
4,A-MIA-UM000463-BR-MIA-201307288,-0.00216,A-MIA-UM000463,1
...,...,...,...,...
10973,A-MIA-UM000846-BR-MIA-200809330,-0.00264,A-MIA-UM000846,1
10974,A-MIA-UM005927-BR-MIA-200521579,-0.00276,A-MIA-UM005927,1
10975,A-MIA-UM001230-BR-MIA-200427223,-0.00264,A-MIA-UM001230,1
10976,A-MIA-UM001640-BR-MIA-201335875,-0.00272,A-MIA-UM001640,1


In [153]:
info.plot.scatter(x="SUBJID",y="diagnosis")

ImportError: matplotlib is required for plotting when the default backend "matplotlib" is selected.